In [ ]:

import argparse

from apiclient.discovery import build
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
CLIENT_SECRETS_PATH = 'client_secret_54298394999-eqvdgc3tked8pu0ec4pj27l73pd171b7.apps.googleusercontent.com.json' # Path to client_secrets.json file.
VIEW_ID = '80590491'


def initialize_analyticsreporting():
  """Initializes the analyticsreporting service object.

  Returns:
    analytics an authorized analyticsreporting service object.
  """
  # Parse command-line arguments.
  parser = argparse.ArgumentParser(
      formatter_class=argparse.RawDescriptionHelpFormatter,
      parents=[tools.argparser])
  flags = parser.parse_args([])

  # Set up a Flow object to be used if we need to authenticate.
  flow = client.flow_from_clientsecrets(
      CLIENT_SECRETS_PATH, scope=SCOPES,
      message=tools.message_if_missing(CLIENT_SECRETS_PATH))

  # Prepare credentials, and authorize HTTP object with them.
  # If the credentials don't exist or are invalid run through the native client
  # flow. The Storage object will ensure that if successful the good
  # credentials will get written back to a file.
  storage = file.Storage('analyticsreporting.dat')
  credentials = storage.get()
  if credentials is None or credentials.invalid:
    credentials = tools.run_flow(flow, storage, flags)
  http = credentials.authorize(http=httplib2.Http())

  # Build the service object.
  analytics = build('analyticsreporting', 'v4', http=http)

  return analytics


In [ ]:
import pandas as pd

# Prior to May 2020 - Only 1 Google Ads campaign 

In [ ]:
def get_report_beforeMay(analytics):
  return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'pageSize': 10000, # over-write limit 1000
          'dateRanges': [{'startDate': '2019-06-01', 'endDate': '2020-05-01'}], # prior to change 
          'dimensions': [{'name': 'ga:keyword'}, 
                         {'name': 'ga:campaign'}, 
                         {'name': 'ga:country'}], #our new dimensions
          'metrics': [{'expression': 'ga:goal4Completions'},
                      {'expression': 'ga:goal5Completions'},
                      {'expression': 'ga:goal6Completions'},
                      {'expression': 'ga:goal7Completions'},
                      {'expression': 'ga:goal8Completions'}, 
                      {'expression': 'ga:sessions'},
                      {'expression': 'ga:bounces'}]
        }]
      }
  ).execute()

analytics_beforeMay = initialize_analyticsreporting()

In [ ]:
def print_response(response):
    data = response.get('reports', [])[0].get('data', {}).get('rows', [])
    df = pd.DataFrame(data)
    ###---- Dimension ----###
    # Parse 1st dimensions into 1st column
    df.insert(loc=0, column="keyword", value=df['dimensions'].apply(lambda x: x[0]))
    # Parse 2nd dimension into 2nd column
    df.insert(loc=1, column="campaign", value=df['dimensions'].apply(lambda x: x[1]))
    # Parse 3nrd dimension into 2nd column
    df['dimensions'] = df['dimensions'].apply(lambda x: x[2])
    
    ###---- Metrics ----###
    df['goal4Completions'] = df['metrics'].apply(lambda x: x[0]).apply(lambda x: x.get('values')).apply(lambda x: x[0])
    df['goal5Completions'] = df['metrics'].apply(lambda x: x[0]).apply(lambda x: x.get('values')).apply(lambda x: x[1])
    df['goal6Completions'] = df['metrics'].apply(lambda x: x[0]).apply(lambda x: x.get('values')).apply(lambda x: x[2])
    df['goal7Completions'] = df['metrics'].apply(lambda x: x[0]).apply(lambda x: x.get('values')).apply(lambda x: x[3])
    df['goal8Completions'] = df['metrics'].apply(lambda x: x[0]).apply(lambda x: x.get('values')).apply(lambda x: x[4])
    df['sessions'] = df['metrics'].apply(lambda x: x[0]).apply(lambda x: x.get('values')).apply(lambda x: x[5])
    df['bounces'] = df['metrics'].apply(lambda x: x[0]).apply(lambda x: x.get('values')).apply(lambda x: x[6])
    
    # Rename columns
    df.rename(columns={'dimensions': 'country'}, inplace=True)
    df.drop(columns = ['metrics'], inplace = True)
    return df


response = get_report_beforeMay(analytics_beforeMay)
before_may = print_response(response) #print our response

In [ ]:
before_may

,keyword,campaign,country,goal4Completions,goal5Completions,goal6Completions,goal7Completions,goal8Completions,sessions,bounces
0,(not provided),(not set),(not set),1,15,15,17,3,34,6
1,(not provided),(not set),Afghanistan,0,1,0,0,0,1,0
2,(not provided),(not set),Albania,0,4,16,14,2,23,2
3,(not provided),(not set),Algeria,1,25,17,25,6,61,16
4,(not provided),(not set),American Samoa,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...
6888,اوبيا,(not set),Sudan,0,0,0,0,0,2,2
6889,اوبيا,(not set),Turkey,0,1,0,1,0,1,0
6890,بحث عن الطاقه الخلويه,(not set),Saudi Arabia,0,1,0,1,0,1,0
6891,تكيف الحيوانات مع البيئة التي تعيش فيها,(not set),Oman,0,1,0,0,0,1,0


In [ ]:
before_may.to_csv('before_may_raw.csv')